In [1]:
import os
import numpy as np
import pandas as pd
import random
import time
import scanpy as sc
import torch

import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO)
import scDualGN

def seed_torch(seed=666):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
seed_torch()

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100)
sc.settings.set_figure_params(dpi_save=300)

In [3]:
#adata = sc.read_h5ad('./imputation_data/Human_ESC_Raw_celltype.h5ad')
adata = sc.read_h5ad('./Human_ESC_Raw_time.h5ad')
adata.obs['celltype'] = 0
adata.obs['celltype_str'] = adata.obs['celltype']

In [4]:
adata

AnnData object with n_obs × n_vars = 350 × 19097
    obs: 'celltype', 'celltype_str'
    layers: 'raw'

In [5]:
sc.pp.normalize_total(adata)

normalizing counts per cell
    finished (0:00:00)


In [6]:
adata

AnnData object with n_obs × n_vars = 350 × 19097
    obs: 'celltype', 'celltype_str'
    layers: 'raw'

In [7]:
#leiden
start = time.time()
z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata,n_z=32,n_epoch_update_pq=5, n_epochs=30,alpha_dualvae=0.02,beta_daulvae=1,gamma_dualvae=4,cluster_alg='leiden',GPU_id=2,n_neighbors=20)
end = time.time()

print('running time = {}'.format(end-start))

INFO:scDualGN.run:device: cuda:2


0.001


INFO:scDualGN.run:pretrain device: cuda:2
INFO:scDualGN.run:start pretraining...
INFO:scDualGN.run:Epoch 0/100,Overall loss:31114209280.0000,MSE:31114190848.0000,MSE1:3995.6558,KL: 7.6262
INFO:scDualGN.run:Epoch 1/100,Overall loss:31112849408.0000,MSE:31112835072.0000,MSE1:3403.7986,KL: 14.2155
INFO:scDualGN.run:Epoch 2/100,Overall loss:31111514112.0000,MSE:31111501824.0000,MSE1:3170.6843,KL: 17.0040
INFO:scDualGN.run:Epoch 3/100,Overall loss:31110207488.0000,MSE:31110195200.0000,MSE1:3281.3496,KL: 18.2192
INFO:scDualGN.run:Epoch 4/100,Overall loss:31108855808.0000,MSE:31108847616.0000,MSE1:3219.0671,KL: 19.3535
INFO:scDualGN.run:Epoch 5/100,Overall loss:31107526656.0000,MSE:31107510272.0000,MSE1:3493.4243,KL: 20.5216
INFO:scDualGN.run:Epoch 6/100,Overall loss:31106187264.0000,MSE:31106174976.0000,MSE1:3481.7114,KL: 22.1578
INFO:scDualGN.run:Epoch 7/100,Overall loss:31104786432.0000,MSE:31104772096.0000,MSE1:3493.4561,KL: 24.1059
INFO:scDualGN.run:Epoch 8/100,Overall loss:31103404032.0

350


INFO:scDualGN.run:Epoch 1/30, Loss - overall: 30901563392.0000,daul_VAE:30901563392.0000,KL:0.0015,Center:386.9293
INFO:scDualGN.run:Epoch 2/30, Loss - overall: 30899728384.0000,daul_VAE:30899728384.0000,KL:0.0020,Center:402.5963
INFO:scDualGN.run:Epoch 3/30, Loss - overall: 30897012736.0000,daul_VAE:30897012736.0000,KL:0.0016,Center:409.6999
INFO:scDualGN.run:Epoch 4/30, Loss - overall: 30894944256.0000,daul_VAE:30894944256.0000,KL:0.0019,Center:415.2747
INFO:scDualGN.run:Epoch 5/30, Loss - overall: 30892609536.0000,daul_VAE:30892609536.0000,KL:0.0018,Center:422.0048
INFO:scDualGN.run:epoch 6/30
INFO:scDualGN.evalution:acc=0.9886, nmi=0.9192, ari=0.9546
INFO:scDualGN.run:Epoch 6/30, Loss - overall: 30890137600.0000,daul_VAE:30890137600.0000,KL:0.0015,Center:427.8775
INFO:scDualGN.run:Epoch 7/30, Loss - overall: 30887776256.0000,daul_VAE:30887776256.0000,KL:0.0015,Center:433.0875
INFO:scDualGN.run:Epoch 8/30, Loss - overall: 30885517312.0000,daul_VAE:30885517312.0000,KL:0.0017,Center:4

running time = 34.78262400627136


In [ ]:
adata.obs['celltype_scDualGN']=y_pred_lab
adata.obsm['X_scDualGN']=z_final